import pandas as pd
import numpy as np

7.1  Create a crawler 

In [ ]:
# %load ../homework7/webcrawling_scraper/webcrawling_scraper/items.py

# Define here the models for your scraped items
#
# See documentation in:
# http://doc.scrapy.org/en/latest/topics/items.html

import scrapy

class WebCrawlingScraperItem(scrapy.Item):
    # The source URL
    url_from = scrapy.Field()
    # The destination URL
    url_to = scrapy.Field()


In [ ]:
# %load ../homework7/webcrawling_scraper/webcrawling_scraper/spiders/datablogger.py
import scrapy
from scrapy.linkextractor import LinkExtractor
from scrapy.spiders import Rule, CrawlSpider
from webcrawling_scraper.items import WebCrawlingScraperItem


class DatabloggerSpider(CrawlSpider):
    custom_settings = {'DEPTH_LIMIT': 200,}
    # The name of the spider
    name = "datablogger"
    # The domains that are allowed (links to other domains are skipped)
    allowed_domains = ["data-blogger.com"]
    # The URLs to start with
    start_urls = ["https://www.data-blogger.com/"]

    # This spider has one rule: extract all (unique and canonicalized) links, follow them and parse them using the parse_items method
    rules = [
        Rule(
            LinkExtractor(
                canonicalize=True,
                unique=True
            ),
            follow=True,
            callback="parse_items"
        )
    ]

    # Method which starts the requests by visiting all URLs specified in start_urls
    def start_requests(self):
        for url in self.start_urls:
            yield scrapy.Request(url, callback=self.parse, dont_filter=True)

    # Method for parsing items
    def parse_items(self, response):
        # The list of items that are found on the particular page
        items = []
        # Only extract canonicalized and unique links (with respect to the current page)
        links = LinkExtractor(canonicalize=True, unique=True).extract_links(response)
        # Now go through all the found links
        for link in links:
            # Check whether the domain of the URL of the link is allowed; so whether it is in one of the allowed domains
            is_allowed = False
            for allowed_domain in self.allowed_domains:
                if allowed_domain in link.url:
                    is_allowed = True
            # If it is allowed, create a new item and add it to the list of found items
            if is_allowed:
                item = WebCrawlingScraperItem()
                item['url_from'] = response.url
                item['url_to'] = link.url
                items.append(item)
        # Return all the found items
        return items

7.2  Create a Stochastic matrix from its resulting crawling

In [29]:
# read the data
data = pd.read_csv("webcrawling_scraper/links.csv")


In [87]:
#grouping
urls_to=data[['url_from','url_to']].groupby(['url_from'],as_index=False).count()

In [88]:
urls_to

,url_from,url_to
0,https://www.data-blogger.com/,47
1,https://www.data-blogger.com/2016/08/18/scrapi...,32
2,https://www.data-blogger.com/2017/02/24/gather...,32
3,https://www.data-blogger.com/2017/06/15/fraud-...,16
4,https://www.data-blogger.com/author/sarena/,28
5,https://www.data-blogger.com/page/2/,45


Now we would need to create the Stostic matrix to perfrom the analyis
<img src="https://cs7083.files.wordpress.com/2013/01/b.png">
Now we would need to create the Stostic matrix to perfrom the analyis
<img src="https://cs7083.files.wordpress.com/2013/01/a.png">

In [102]:
H=np.zeros((6,6))


In [103]:
H

array([[ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.]])

In [104]:
# creating Stochastic matrix
from random import randint
for i in range (5):
    x=1/float(urls_to.loc[[i]]['url_to'])   
    H[i][randint(0, 5)]=x

In [105]:
H

array([[ 0.        ,  0.0212766 ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.03125   ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.03125   ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.0625    ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.03571429],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ]])

7.3 Pass it through the Page Rank algorithm and provide the list of the top 5 page URLs in your sample

In [106]:
from numpy import *
 
def pagerank(H):
    n= len(H)
    w = zeros(n)
    rho = 1./n * ones(n);
    for i in range(n):
      if multiply.reduce(H[i]== zeros(n)):
        w[i] = 1
    newH = H + outer((1./n * w),ones(n))
 
    theta=0.85
    G = (theta * newH) + ((1-theta) * outer(1./n * ones(n), ones(n)))
    print (rho)
    for j in range(10):
        rho = dot(rho,G)
        print (rho)
        if j==9:
            return rho

In [107]:
res=pagerank(H)

[ 0.16666667  0.16666667  0.16666667  0.16666667  0.16666667  0.16666667]
[ 0.05303819  0.05605238  0.05746528  0.04861111  0.04861111  0.05367063]
[ 0.01706598  0.01798765  0.01812202  0.01553956  0.01553956  0.01701525]
[ 0.00542361  0.00572868  0.00576778  0.00494224  0.00494224  0.00541398]
[ 0.00172565  0.0018227   0.001835    0.00157244  0.00157244  0.00172248]
[ 0.00054903  0.00057991  0.00058382  0.00050029  0.00050029  0.00054802]
[ 0.00017468  0.0001845   0.00018575  0.00015917  0.00015917  0.00017436]
[  5.55751637e-05   5.87011694e-05   5.90971467e-05   5.06412422e-05
   5.06412422e-05   5.54731877e-05]
[  1.76816983e-05   1.86762629e-05   1.88022464e-05   1.61119304e-05
   1.61119304e-05   1.76492538e-05]
[  5.62557868e-06   5.94200764e-06   5.98209031e-06   5.12614401e-06
   5.12614401e-06   5.61525619e-06]
[  1.78982442e-06   1.89049891e-06   1.90325155e-06   1.63092515e-06
   1.63092515e-06   1.78654023e-06]


In [113]:
urls_to.loc[np.argsort(res)[-10:]]['url_from']

3    https://www.data-blogger.com/2017/06/15/fraud-...
4          https://www.data-blogger.com/author/sarena/
5                 https://www.data-blogger.com/page/2/
0                        https://www.data-blogger.com/
1    https://www.data-blogger.com/2016/08/18/scrapi...
2    https://www.data-blogger.com/2017/02/24/gather...
Name: url_from, dtype: object

Reference:
https://www.data-blogger.com/2016/08/18/scraping-a-website-with-python-scrapy/
